In [62]:
#
# SDSS DR6 data in GAMA Survey - Table: InputCatA
#
# From http://skyserver.sdss.org/dr7/en/help/docs/realquery.asp#egal
#
#SELECT ObjID
#FROM Galaxy as G
#WHERE
#    G.lnlDev_r > G.lnlExp_r + 0.1 
#    -- the likelihood of the deVaucouleours profile fit is 10% greater than the 
#    -- likelihood of the exponential fit 
#    and G.lnlExp_r > -999 
#    -- and the likelihoods are actually meaningful 
#    and (G.flags & (dbo.fPhotoFlags('BINNED1') + dbo.fPhotoFlags('BINNED2') + 
#    dbo.fPhotoFlags('BINNED4'))) > 0 
#    -- and it is detected from at least one of the binned images 
#    and (G.flags & ( dbo.fPhotoFlags('BLENDED') + dbo.fPhotoFlags('NODEBLEND') + 
#    dbo.fPhotoFlags('CHILD'))) != dbo.fPhotoFlags('BLENDED') 
#    -- and, if it is blended, it is either a child or not deblended further 
#    and (G.flags & (dbo.fPhotoFlags('EDGE') + dbo.fPhotoFlags('SATURATED'))) = 0 
#    -- and it is not near a ccd edge or saturated, where measurements may be bad 
#    and G.petroMag_i > 17.5 
#    -- and it is fainter than 17.5 in i-band 
#    and (G.petroMag_r > 15.5 or G.petroR50_r > 2) 
#    and (G.petroMag_r > 0 and G.g > 0 and G.r > 0 and G.i > 0) 
#    and ( (G.petroMag_r - G.extinction_r) < 19.2 
#    and (G.petroMag_r - G.extinction_r < 
#    (13.1 + (7/3)*(G.g - G.r) + 4 *(G.r - G.i) - 4 * 0.18) ) 
#    and ( (G.r - G.i - (G.g - G.r)/4 - 0.18) < 0.2 ) 
#    and ( (G.r - G.i - (G.g - G.r)/4 - 0.18) > -0.2 ) ) 
#    or ( (G.petroMag_r - G.extinction_r < 19.5) 
#    and ( (G.r - G.i - (G.g - G.r)/4 - 0.18) > 
#    (0.45 - 4*(G.g - G.r) ) ) 
#    and ( (G.g - G.r) > (1.35 + 0.25 *(G.r - G.i) ) ) ) 
#    -- and many constraints on colors and mags to make it have elliptical-type colors.

# Need to check if MODELMAG_R = G.r

# cn Column Number for name
def cn(table, name) :
    return(table.colnames.index(name))

import numpy as np
from astropy.table import Table, vstack, join
t = Table.read('GAMA_Data/InputCatA.fits')
print(t.colnames)
#ta = np.array(t)
print('Source Table : '+str(len(t)))
n = t[t['PETROMAG_I'] > 17.5] # fainter than 17.5 in i-band
# and (G.petroMag_r > 15.5 or G.petroR50_r > 2)
n1 = n[np.logical_and(n['PETROMAG_R'] > 15.5,n['PETROR50_R'] > 2)]
# and (G.petroMag_r > 0 and G.g > 0 and G.r > 0 and G.i > 0) 
n2 = n1[np.logical_and(n1['PETROMAG_R'] > 0,n1['MODELMAG_G'] > 0,n1['MODELMAG_I'] > 0)]
# and ( (G.petroMag_r - G.extinction_r) < 19.2
n3 = n2[(n2['PETROMAG_R'] - n2['EXTINCTION_R']) < 19.2]
#and (G.petroMag_r - G.extinction_r < 
#    (13.1 + (7/3)*(G.g - G.r) + 4 *(G.r - G.i) - 4 * 0.18) )
n4 = n3[(n3['PETROMAG_R'] - n3['EXTINCTION_R']) < (13.1 + (7/3)*(n3['MODELMAG_G']-n3['MODELMAG_R'])+4*(n3['MODELMAG_R']-n3['MODELMAG_I']) - 4*0.18)]
#    and ( (G.r - G.i - (G.g - G.r)/4 - 0.18) < 0.2 )
n5 = n4[(n4['MODELMAG_R'] - n4['MODELMAG_I'] - (n4['MODELMAG_G'] -n4['MODELMAG_R'])/4 - 0.18) < 0.2]
# and ( (G.r - G.i - (G.g - G.r)/4 - 0.18) > -0.2 ) ) 
n6 = n5[(n5['MODELMAG_R'] - n5['MODELMAG_I'] - (n5['MODELMAG_G'] -n5['MODELMAG_R'])/4 - 0.18) < -0.2]
# AS an OR need to Stack tables
# or ( (G.petroMag_r - G.extinction_r < 19.5) 
n7 = n[n['PETROMAG_R'] - n['EXTINCTION_R'] < 19.5]
n8 = vstack([n6,n7])
# and ( (G.r - G.i - (G.g - G.r)/4 - 0.18) > 
#    (0.45 - 4*(G.g - G.r) ) ) 
n9 = n8[n8['MODELMAG_R']-n8['MODELMAG_I']-(n8['MODELMAG_G']- n8['MODELMAG_R'])/4 - 0.18 > (0.45 - 4*(n8['MODELMAG_G']- n8['MODELMAG_R']))]
# and ( (G.g - G.r) > (1.35 + 0.25 *(G.r - G.i) ) ) )
na = n9[n9['MODELMAG_G']- n9['MODELMAG_R'] > ( 1.35 + 0.25 * (n9['MODELMAG_R'] - n9['MODELMAG_I']))]
print('Table lengths')
print('n  : '+str(len(n)))
print('n1 : '+str(len(n1)))
print('n2 : '+str(len(n2)))
print('n3 : '+str(len(n3)))
print('n4 : '+str(len(n4)))
print('n5 : '+str(len(n5)))
print('n6 : '+str(len(n6)))
print('n7 : '+str(len(n7)))
print('n8 : '+str(len(n8)))
print('n9 : '+str(len(n9)))
print('na : '+str(len(na)))
#print(n1)

stellarMasses = Table.read('GAMA_Data/StellarMasses.fits')
sdss_elliptical = join(na,stellarMasses,keys='CATAID',join_type='inner')
sdss_red_ellipticalL = sdss_elliptical[sdss_elliptical['uminusr'] > 1.8]
sdss_red_ellipticalH = sdss_elliptical[sdss_elliptical['uminusr'] < 2.8]
sdss_red_elliptical.sort('uminusr')
print('sdss_red_elliptical : '+str(len(sdss_red_elliptical)))
sdss_red_elliptical.write('GAMA_Data/SDSS_Red_Elliptical.fits', format='fits',overwrite=True)


['CATAID', 'OBJID', 'RA', 'DEC', 'FLAGS', 'PRIMTARGET', 'PETRORAD_R', 'PSFMAG_R', 'FIBERMAG_R', 'FLAGS_R', 'EXTINCTION_R', 'PETROR90_R', 'PETROR50_R', 'PETROMAG_U', 'PETROMAG_G', 'PETROMAG_R', 'PETROMAG_I', 'PETROMAG_Z', 'MODELMAG_U', 'MODELMAG_G', 'MODELMAG_R', 'MODELMAG_I', 'MODELMAG_Z', 'STATUS', 'RUN']
Source Table : 960510
Table lengths
n  : 851165
n1 : 118831
n2 : 118825
n3 : 66082
n4 : 5753
n5 : 4701
n6 : 1171
n7 : 445357
n8 : 446528
n9 : 440239
na : 65934


sdss_red_elliptical : 9085
